In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pymongo
import time
import random
import pdb


### Make a list of the urls of the categories in the forum

In [3]:
user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
html_page = requests.get('https://curltalk.naturallycurly.com/', headers = {'User-Agent': user_agent} ) #Make a get request to retrieve the page
soup = BeautifulSoup(html_page.content, 'html.parser')
link_list_forum_top = []
for link in soup.find_all('a'):
    link_list_forum_top.append(link.get('href'))
categories = [s for s in link_list_forum_top if (("categories" in s) and ('https' in s))]
categories[0:]

['https://curltalk.naturallycurly.com/categories/announcements',
 'https://curltalk.naturallycurly.com/categories/newbies-info',
 'https://curltalk.naturallycurly.com/categories/introductions',
 'https://curltalk.naturallycurly.com/categories/curlmart-news',
 'https://curltalk.naturallycurly.com/categories/feedback',
 'https://curltalk.naturallycurly.com/categories/c2',
 'https://curltalk.naturallycurly.com/categories/c3',
 'https://curltalk.naturallycurly.com/categories/c4',
 'https://curltalk.naturallycurly.com/categories/general-discussion-about-curly-hair',
 'https://curltalk.naturallycurly.com/categories/salon-stylist-chatter',
 'https://curltalk.naturallycurly.com/categories/transitioning',
 'https://curltalk.naturallycurly.com/categories/thin-thinning-hair',
 'https://curltalk.naturallycurly.com/categories/readers-product-reviews',
 'https://curltalk.naturallycurly.com/categories/hall-of-shame-discussion',
 'https://curltalk.naturallycurly.com/categories/hall-of-fame-discussion'

#### Get the discussion topic urls for the first x pages of the category

In [210]:
# %%time

link_listdiscussion = []
for i in range(0, 100):
    url = f'{categories[46]}/p{i}'
    print(url)
    user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
    html_page = requests.get(url, headers = {'User-Agent': user_agent} ) #Make a get request to retrieve the page
     
    # Check status code for an appropriate response fromt the API
    status = html_page.status_code
    if status != 200:
        print(f'Error improper response code. Code is {status}')
        #break
    soup = BeautifulSoup(html_page.content, 'html.parser') #Pass the page contents to beautiful soup for parsing
    page1_of_discussion = []
    #time.sleep(random.randint(20,45))
    for link in soup.find_all('a'):
        link_listdiscussion.append(link.get('href'))
        topics = [s for s in link_listdiscussion if (("/discussion" in s) and ('https' in s))]

print(len(topics))     
# Loop through all the topics found for each catagory        
list_for_mongo = []
count = 1
for topic in topics:
    url2 = topic
    user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
    html_page2 = requests.get(url2, headers = {'User-Agent': user_agent} )
    
    if status != 200:
        print(f'Error improper response code. Code is {status}')
        #break
        
    soup2 = BeautifulSoup(html_page2.content, 'html.parser')
    signatures = soup2.find_all('div', class_="Signature UserSignature userContent")
    
    for i in range(0,len(signatures)):
        sig = {}
        sig['signature'] = soup2.find_all('div', class_="Signature UserSignature userContent")[i].get_text()
        list_for_mongo.append(sig)
#     time.sleep(random.randint(1,5))
    print(count)
    count += 1
    

https://curltalk.naturallycurly.com/categories/recipes-stuff-to-eat/p0
https://curltalk.naturallycurly.com/categories/recipes-stuff-to-eat/p1
https://curltalk.naturallycurly.com/categories/recipes-stuff-to-eat/p2
https://curltalk.naturallycurly.com/categories/recipes-stuff-to-eat/p3
https://curltalk.naturallycurly.com/categories/recipes-stuff-to-eat/p4
https://curltalk.naturallycurly.com/categories/recipes-stuff-to-eat/p5
https://curltalk.naturallycurly.com/categories/recipes-stuff-to-eat/p6
https://curltalk.naturallycurly.com/categories/recipes-stuff-to-eat/p7
https://curltalk.naturallycurly.com/categories/recipes-stuff-to-eat/p8
https://curltalk.naturallycurly.com/categories/recipes-stuff-to-eat/p9
https://curltalk.naturallycurly.com/categories/recipes-stuff-to-eat/p10
https://curltalk.naturallycurly.com/categories/recipes-stuff-to-eat/p11
https://curltalk.naturallycurly.com/categories/recipes-stuff-to-eat/p12
https://curltalk.naturallycurly.com/categories/recipes-stuff-to-eat/p13
ht

https://curltalk.naturallycurly.com/categories/recipes-stuff-to-eat/p82
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/recipes-stuff-to-eat/p83
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/recipes-stuff-to-eat/p84
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/recipes-stuff-to-eat/p85
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/recipes-stuff-to-eat/p86
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/recipes-stuff-to-eat/p87
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/recipes-stuff-to-eat/p88
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/recipes-stuff-to-eat/p89
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/recipes-stuff-to-eat/p90
Error improper r

136
Error improper response code. Code is 404
137
Error improper response code. Code is 404
138
Error improper response code. Code is 404
139
Error improper response code. Code is 404
140
Error improper response code. Code is 404
141
Error improper response code. Code is 404
142
Error improper response code. Code is 404
143
Error improper response code. Code is 404
144
Error improper response code. Code is 404
145
Error improper response code. Code is 404
146
Error improper response code. Code is 404
147
Error improper response code. Code is 404
148
Error improper response code. Code is 404
149
Error improper response code. Code is 404
150
Error improper response code. Code is 404
151
Error improper response code. Code is 404
152
Error improper response code. Code is 404
153
Error improper response code. Code is 404
154
Error improper response code. Code is 404
155
Error improper response code. Code is 404
156
Error improper response code. Code is 404
157
Error improper response code. 

Error improper response code. Code is 404
316
Error improper response code. Code is 404
317
Error improper response code. Code is 404
318
Error improper response code. Code is 404
319
Error improper response code. Code is 404
320
Error improper response code. Code is 404
321
Error improper response code. Code is 404
322
Error improper response code. Code is 404
323
Error improper response code. Code is 404
324
Error improper response code. Code is 404
325
Error improper response code. Code is 404
326
Error improper response code. Code is 404
327
Error improper response code. Code is 404
328
Error improper response code. Code is 404
329
Error improper response code. Code is 404
330
Error improper response code. Code is 404
331
Error improper response code. Code is 404
332
Error improper response code. Code is 404
333
Error improper response code. Code is 404
334
Error improper response code. Code is 404
335
Error improper response code. Code is 404
336
Error improper response code. Code

Error improper response code. Code is 404
495
Error improper response code. Code is 404
496
Error improper response code. Code is 404
497
Error improper response code. Code is 404
498
Error improper response code. Code is 404
499
Error improper response code. Code is 404
500
Error improper response code. Code is 404
501
Error improper response code. Code is 404
502
Error improper response code. Code is 404
503
Error improper response code. Code is 404
504
Error improper response code. Code is 404
505
Error improper response code. Code is 404
506
Error improper response code. Code is 404
507
Error improper response code. Code is 404
508
Error improper response code. Code is 404
509
Error improper response code. Code is 404
510
Error improper response code. Code is 404
511
Error improper response code. Code is 404
512
Error improper response code. Code is 404
513
Error improper response code. Code is 404
514
Error improper response code. Code is 404
515
Error improper response code. Code

Error improper response code. Code is 404
674
Error improper response code. Code is 404
675
Error improper response code. Code is 404
676
Error improper response code. Code is 404
677
Error improper response code. Code is 404
678
Error improper response code. Code is 404
679
Error improper response code. Code is 404
680
Error improper response code. Code is 404
681
Error improper response code. Code is 404
682
Error improper response code. Code is 404
683
Error improper response code. Code is 404
684
Error improper response code. Code is 404
685
Error improper response code. Code is 404
686
Error improper response code. Code is 404
687
Error improper response code. Code is 404
688
Error improper response code. Code is 404
689
Error improper response code. Code is 404
690
Error improper response code. Code is 404
691
Error improper response code. Code is 404
692
Error improper response code. Code is 404
693
Error improper response code. Code is 404
694
Error improper response code. Code

In [211]:
len(list_for_mongo)

3410

### Create a mongoDB and have data stored from scrape 

In [212]:
# DB creation 
myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
curlydb = myclient['Naturaly_Curly_db']
curly_collection = curlydb['curly_profiles']

In [213]:
entry2 = curly_collection.insert_many(list_for_mongo)

In [214]:
query_1 = curly_collection.find({})
test = []
for x in query_1:
    test.append(x['signature'])
    #print(x)
print(len(test))    

469904


In [215]:
test_set = set(test)

In [216]:
# Number of unique entries so far
len(test_set)
#7789

7789